## Programming Assignment #4

**Data Science Practicum**

100 points possible.

This assignment will ask you to develop a notebook requiring various data-related activities (importing, cleaning, manipulating, merging, and aggregating).

# Topic: Measuring Neighborhood Availability of Alcohol

Use data from KY’s Department of Alcoholic Beverage Control (ABC) to calculate the availability of alcohol in Fayette county’s neighborhoods.  In particular for each neighborhood, **calculate the rate of liquor licenses per capita (the number of licenses divided by the neighborhood’s population).  Show the top 20 neighborhoods with the highest rate of alcohol availability. Show the top 20 neighborhoods with the highest number of licenses.  Discuss whether or not these two top-20 lists differ and how.**  Define neighborhood as a US Census Bureau’s tract (discussed below).  Present this as a notebook, complete with documentation and code. Use the notebook to communicate about the data, process, and results.

# Part 0 -- Submission Details


(10 points) Please enter your name and the date below. Submit your answers as a completed notebook by the deadline posted on Canvas.  Late submissions will not get credit for this section.

Name: Duong Hoang

Date: 10/27/2022

# Part 1 -- Data Import

(10 points) Are the appropriate data sets imported correctly?

In [4]:
import pandas as pd
import requests

# read the small license data file provided in the assignment description
# excluding the first 3 lines (non-data)
license_data = pd.read_csv('ActiveLicenses-Small.csv', skiprows=3)

# get population data from the US Census Bureau’s API
population_data = pd.DataFrame(requests.get(url='https://api.census.gov/data/2020/acs/acs5/profile?get=NAME,DP05_0001E&for=tract:*&in=state:21&in=county:067').json()) 
# handling misreading the blank line as header
population_data.rename(columns=population_data.iloc[0], inplace=True)
population_data.drop(population_data.index[0], inplace=True)
population_data.reset_index(drop=True, inplace=True)
population_data['tract'] = pd.to_numeric(population_data['tract'], downcast='unsigned', errors='coerce')

# output two datasets
print('License Dataset:')
print(license_data)
print('\nTract Population Dataset:')
print(population_data)

License Dataset:
     SiteID   County       City                         DBA  \
0     22790  Fayette  Lexington  21c Museum Hotel Lexington   
1     26561  Fayette  Lexington      A Cup of Common Wealth   
2      5119  Fayette  Lexington   A P Suggins Bar And Grill   
3     28033  Fayette  Lexington      Agave & Rye Epic Tacos   
4     19206  Fayette  Lexington               Agave and Rye   
..      ...      ...        ...                         ...   
496   24014  Fayette  Lexington               World of Beer   
497   15919  Fayette  Lexington                        XOXO   
498   28812  Fayette  Lexington          Yesterday's Lounge   
499    5944  Fayette  Lexington            Zen Sushi & sake   
500   27204  Fayette  Lexington                  Zim's Cafe   

                            Licensee                   PremisesStreet  \
0                167 MAIN HOTEL LLC                     167 W Main St   
1        A Cup of Common Wealth LLC                   105 Eastern Ave   
2      

# Part 2 -- Data Manipulation

(20 points) Does the raw data get translated into usable data? (for example, geocoding)

In [5]:
# format address for request
req_address = license_data[['PremisesStreet', 'PremisesCityState']].apply(', '.join, axis=1)
req_address = req_address.str.replace(' ', '+')
req_address = req_address.str.replace(',', '%2C')
# create request to geocoding the license data
geocoding = req_address.apply(lambda address: requests.get('https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?address='+address+'&benchmark=Public_AR_Census2020&vintage=Census2020_Census2020&layers=6&format=json'))

In [6]:
# extract tract name from json responses
tract_names = []
for i in range(geocoding.size):
    try:
        # get tract name from good response
        tract_name = geocoding[i].json()['result']['addressMatches'][0]['geographies']['Census Tracts'][0]['NAME']
        tract_names.append(tract_name)
    except:
        # append NA for bad response
        tract_names.append(pd.NA)

# append tract names to the corresponding license record
license_data['TractName'] = tract_names

# drop all the entries that failed to be processed (where tract name is NA value)
license_data.dropna(subset=['TractName'], inplace=True)
license_data.reset_index(drop=True, inplace=True)

# output license data with corresponding tract names
print(license_data)

     SiteID   County       City                         DBA  \
0     22790  Fayette  Lexington  21c Museum Hotel Lexington   
1     26561  Fayette  Lexington      A Cup of Common Wealth   
2      5119  Fayette  Lexington   A P Suggins Bar And Grill   
3     28033  Fayette  Lexington      Agave & Rye Epic Tacos   
4     19206  Fayette  Lexington               Agave and Rye   
..      ...      ...        ...                         ...   
412    3355  Fayette  Lexington         Windy Corner Market   
413   13143  Fayette  Lexington             Wine and Market   
414   15919  Fayette  Lexington                        XOXO   
415    5944  Fayette  Lexington            Zen Sushi & sake   
416   27204  Fayette  Lexington                  Zim's Cafe   

                            Licensee              PremisesStreet  \
0                167 MAIN HOTEL LLC                167 W Main St   
1        A Cup of Common Wealth LLC              105 Eastern Ave   
2          ROMANY RESTAURANTS, INC.    

# Part 3 -- Rate Calculation

(20 points) Is the rate calculated correctly? Are both data sets utilized correctly?

In [7]:
# group records by tract name
neighbors_data = license_data.groupby(['TractName'], dropna=True)
# for each tract, count number of license and get the population size 
# then append to tract dataset
tract_data = []
for name, group in neighbors_data:
    tract_name = name + ', Fayette County, Kentucky'
    population_info =  population_data[population_data['NAME'].str.fullmatch(tract_name)]
    tract_data.append([name, group.shape[0], population_info['tract'].iloc[0]])
tract_data = pd.DataFrame(tract_data, columns=['TractName', 'LicenseCount', 'PopulationSize'])

# calculate the rate of alcohol availability per 1000 people 
tract_data['Rate'] = tract_data['LicenseCount'] / tract_data['PopulationSize'] * 1000

# output tracts and their rates per 1000 people
print(tract_data.loc[:, ['TractName', 'Rate']])

            TractName        Rate
0   Census Tract 1.01  633.663366
1   Census Tract 1.02  166.666667
2     Census Tract 10   17.000000
3     Census Tract 11    0.909091
4     Census Tract 13    3.846154
..                ...         ...
60     Census Tract 6   10.000000
61     Census Tract 7   17.142857
62  Census Tract 8.01    1.248439
63  Census Tract 8.02    2.493766
64     Census Tract 9   27.777778

[65 rows x 2 columns]


# Part 4 -- Total Number Calculation

(10 points) Is the total number of licenses per neighborhood aggregated correctly?

In [8]:
# number of licenses per tract is counted in part 3 using groupby tract name
# output tracts and their license count
print(tract_data.loc[:, ['TractName', 'LicenseCount']])

            TractName  LicenseCount
0   Census Tract 1.01            64
1   Census Tract 1.02            17
2     Census Tract 10            17
3     Census Tract 11             1
4     Census Tract 13             5
..                ...           ...
60     Census Tract 6             6
61     Census Tract 7            12
62  Census Tract 8.01             1
63  Census Tract 8.02             2
64     Census Tract 9            25

[65 rows x 2 columns]


# Part 5 -- Top 20 lists and Discussion

(20 points) Are the two top-20 lists presented correctly and are the results discussed?

In [9]:
# get the top 20 by availability rate per capita and license count
top_20_by_rate = tract_data.nlargest(20, columns='Rate')
top_20_by_count = tract_data.nlargest(20, columns='LicenseCount')

# output those two top-20
print('The top 20 tracts with the highest rate of alcohol availability are:')
print(top_20_by_rate)
print('\nThe top 20 tracts with the highest number of licenses are:')
print(top_20_by_count)

# these two top-20 are different from each other as the number of licenses and 
# the population size of each tract is not propotional throughout the whole dataset, 
# therefore, the rate for each tract will not be propotional to the license count 
# throughout the whole dataset either.
# two tracts (1.02 and 10) with the same license counts (17) but have two different 
# population sizes (102 and 1000) would result in two different rates (166.67 and 17), 
# thus, result in two different rankings by rate while having the same ranking by count

The top 20 tracts with the highest rate of alcohol availability are:
             TractName  LicenseCount  PopulationSize        Rate
0    Census Tract 1.01            64             101  633.663366
1    Census Tract 1.02            17             102  166.666667
10      Census Tract 2            12             200   60.000000
64      Census Tract 9            25             900   27.777778
61      Census Tract 7            12             700   17.142857
2      Census Tract 10            17            1000   17.000000
19      Census Tract 3             5             300   16.666667
59      Census Tract 5             7             500   14.000000
60      Census Tract 6             6             600   10.000000
54  Census Tract 42.04            34            4204    8.087536
8      Census Tract 18            11            1800    6.111111
17     Census Tract 28            16            2800    5.714286
39  Census Tract 39.08            20            3908    5.117707
48      Census Tract 

# Part 6 -- Documentation

(10 points)  Does your notebook communicate about the data, process, and results? Use the format to tell your code’s story.